[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/machinelearnear/vtoonify-in-studio-lab/blob/main/inference_playground.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/machinelearnear/vtoonify-in-studio-lab/blob/main/inference_playground.ipynb)

# Run `VToonify`
- Original: https://github.com/williamyang1991/VToonify
- Modified from [pixel2style2pixel](https://github.com/eladrich/pixel2style2pixel/blob/master/notebooks/inference_playground.ipynb)

## Setup environment

In [1]:
import os
from os.path import exists as path_exists

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
CODE_DIR = 'VToonify'
device = 'cuda'

In [3]:
if not path_exists(CODE_DIR):
    !git clone https://github.com/williamyang1991/VToonify.git $CODE_DIR

In [4]:
if not path_exists('ninja-linux.zip'):
    !wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
    !sudo unzip ninja-linux.zip -d /usr/local/bin/
    !sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force 

try:
    import wget
except ImportError:
    !pip install wget

In [5]:
os.chdir(f'./{CODE_DIR}')
MODEL_DIR = os.path.join(os.path.dirname(os.getcwd()), CODE_DIR, 'checkpoint')
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), CODE_DIR, 'data')
OUT_DIR = os.path.join(os.path.dirname(os.getcwd()), CODE_DIR, 'output')

### Import libraries

In [6]:
try:
    import dlib
except ImportError:
    !pip install cmake==3.21.0
    !pip install dlib==19.21.0

  Using cached cmake-3.21.0-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (19.8 MB)
  Using cached dlib-19.21.0.tar.gz (3.2 MB)
  Created wheel for dlib: filename=dlib-19.21.0-cp38-cp38-linux_x86_64.whl size=4063007 sha256=7dc975dcad0627f6d619d3e31af696f96804e8a11829fc7a22dad84c02036cb4
  Stored in directory: /home/studio-lab-user/.cache/pip/wheels/3a/4e/b6/77346839e430150a62d9b46bf7e0a37181fe01fd07d5d452a7
Successfully built dlib


In [ ]:
def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory ../checkpoint/. """
    current_directory = os.getcwd()
    save_path = MODEL_DIR
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url

MODEL_PATHS = {
    "encoder": {"id": "1NgI4mPkboYvYw3MWcdUaQhkr0OWgs9ej", "name": "encoder.pt"},
    "faceparsing": {"id": "1jY0mTjVB8njDh6e0LP_2UxuRK3MnjoIR", "name": "faceparsing.pth"},
}

MODEL_PATHS_D = {
    "arcane_exstyle": {"id": "1TC67wRJkdmNRZTqYMUEFkrhWRKKZW40c", "name": "exstyle_code.npy"},
    "caricature_exstyle": {"id": "1xr9sx_WmRYJ4qHGTtdVQCSxSo4HP3-ip", "name": "exstyle_code.npy"},
    "cartoon_exstyle": {"id": "1BuCeLk3ASZcoHlbfT28qNru4r5f-hErr", "name": "exstyle_code.npy"},
    "pixar_exstyle": {"id": "1yTaKuSrL7I0i0RYEEK5XD6GI-y5iNUbj", "name": "exstyle_code.npy"},
    "arcane000": {"id": "1pF4fJ8acmawMsjjXo4HXRIOXeZR8jLVh", "name": "generator.pt"},
    "arcane077": {"id": "16rLTF2oC0ZeurnM6hjrfrc8BxtW8P8Qf", "name": "generator.pt"},
    "caricature039": {"id": "1C1E4WEoDWzl0nAxR9okKffFmlMOENbeF", "name": "generator.pt"},
    "caricature068": {"id": "1B1ko1x8fX2aJ4BYCL12AnknVAi3qQc8W", "name": "generator.pt"},
    "cartoon026": {"id": "1YJYODh_vEyUrL0q02okjcicpJhdYY8An", "name": "generator.pt"},
    "cartoon299": {"id": "101qMUMfcI2qDxEbfCBt5mOg2aSqdTaIt", "name": "generator.pt"},
    "pixar052": {"id": "16j_l1x0DD0PjwO8YdplAk69sh3-v95rr", "name": "generator.pt"},
    "cartoon": {"id": "11s0hwhZWTLacMAzZH4OU-o3Qkp54h30J", "name": "generator.pt"},
}

MODEL_PATHS_T = {
    "cartoon": {"id": "1FFtTVgiDKZ_InnwUJLDuA1wfghZp41nX", "name": "vtoonify.pt"},
    "caricature": {"id": "1ReRxttV-macgV3epC61qg4TQ3FGAhGqG", "name": "vtoonify.pt"},
    "arcane": {"id": "1OXU95BOCCT0f6pGbwQ4yQ1EHb2LPd2yb", "name": "vtoonify.pt"},
    "comic": {"id": "1KvawsOXzKgwDM3Z27sagO_KGE_Kc5GZS", "name": "vtoonify.pt"},
    "pixar": {"id": "1pF4fJ8acmawMsjjXo4HXRIOXeZR8jLVh", "name": "vtoonify.pt"},
}

In [ ]:
%matplotlib inline

### Download pre-trained base models 
As part of this repository, we provide pretrained models. We'll download the model and save them to the folder `../checkpoint/`.

In [ ]:
# download pSp encoder and face parsinf network
path = MODEL_PATHS["encoder"]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"])
!{download_command}
path = MODEL_PATHS["faceparsing"]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"])
!{download_command}

## Style Transfer with `VToonify-D`

Transfer a default cartoon style onto a default face image `./data/077436.jpg`:

`python style_transfer.py --scale_image`

The results are saved in the folder `./output/`, where `077436_input.jpg` is the rescaled input image to fit VToonify (this image can serve as the input without `--scale_image`) and `077436_vtoonify_d.jpg` is the result.

Specify the content image and the model, control the style with the following options:

- `--content`: path to the target face image or video
- `--style_id`: the index of the style image (find the mapping between index and the style image here).
- `--style_degree` (default: 0.5): adjust the degree of style.
- `--color_transfer` (default: False): perform color transfer if loading a VToonify-Dsdc model.
- `--ckpt`: path of the VToonify-D model. By default, a VToonify-Dsd trained on cartoon style is loaded.
- `--exstyle_path`: path of the extrinsic style code. By default, codes in the same directory as --ckpt are loaded.
- `--scale_image`: rescale the input image/video to fit VToonify (highly recommend).
- `--padding` (default: 200, 200, 200, 200): left, right, top, bottom paddings to the eye center.
Here is an example of arcane style transfer:

**Style transfer for images**
```sh
python style_transfer.py --content ./data/038648.jpg \
       --scale_image --style_id 77 --style_degree 0.5 \
       --ckpt ./checkpoint/vtoonify_d_arcane/vtoonify_s_d.pt \
       --padding 600 600 600 600     # use large padding to avoid cropping the image
```

Specify `--video` to perform video toonification:

```sh
python style_transfer.py --scale_image --content ./data/YOUR_VIDEO.mp4 --video
```

The above style control options (`--style_id`, `--style_degree`, `--color_transfer`) also work for videos.

In [ ]:
# cartoon026:      balanced 
# cartoon299:      big eyes 
# arcane000:       for female 
# arcane077:       for male 
# pixar052:                  
# caricature039:   big mouth 
# caricature068:   balanced
# style_type = "cartoon026" # ["cartoon026", "cartoon299", "arcane000", "arcane077", "pixar052", "caricature039", "caricature068"]

In [ ]:
# python style_transfer.py --scale_image --content ./data/YOUR_VIDEO.mp4 --video

## Style Transfer with `VToonify-T`

Specify `--backbone` as ''toonify'' to load and use a VToonify-T model.

```sh
python style_transfer.py --content ./data/038648.jpg \
       --scale_image --backbone toonify \
       --ckpt ./checkpoint/vtoonify_t_arcane/vtoonify.pt \
       --padding 600 600 600 600     # use large padding to avoid cropping the image
```

In VToonify-T, `--style_id`, `--style_degree`, `--color_transfer`, `--exstyle_path` are not used.

As with VToonify-D, specify `--video` to perform video toonification.

In [ ]:
style_type = "arcane" # ["cartoon", "caricature", "arcane", "comic", "pixar"]

In [ ]:
# get pre-trained weights
path = MODEL_PATHS_T[style_type]
style_folder = os.path.join(MODEL_DIR, f'vtoonify_t_{style_type}')

if not path_exists(style_folder):
    os.makedirs(style_folder)
download_command = get_download_model_command(file_id=path["id"], file_name = os.path.join(f'vtoonify_t_{style_type}',path["name"]))
!{download_command}

In [ ]:
!python style_transfer.py --content ./data/input.mp4 \
       --scale_image --backbone toonify \
       --ckpt ./checkpoint/vtoonify_t_arcane/vtoonify.pt \
       --padding 600 600 600 600     # use large padding to avoid cropping the image